## Data Loading

In [ ]:
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image

In [ ]:
# Load Annotation dataset
!gdown -- https://drive.google.com/uc?id=1ig7EWh2Zzc3xPS-sR6UWDpcFYwNYLDcL

Downloading...
From (original): https://drive.google.com/uc?id=1ig7EWh2Zzc3xPS-sR6UWDpcFYwNYLDcL
From (redirected): https://drive.google.com/uc?id=1ig7EWh2Zzc3xPS-sR6UWDpcFYwNYLDcL&confirm=t&uuid=ea82c8d4-2777-4b68-9acc-2fee69c13881
To: /content/x_ray_dataset.zip
100% 58.7M/58.7M [00:03<00:00, 18.1MB/s]


In [ ]:
# Unzip
!unzip -qq "/content/x_ray_dataset.zip"

In [ ]:
directory = '/content/x_ray_dataset/annotation_file/images'
num_files = len(os.listdir(directory))
print("Number of files in the directory:", num_files)

Number of files in the directory: 300


In [ ]:
import glob

files = glob.glob(os.path.join(directory, '*.bmp'))

for name in files:
    if not os.path.isdir(name):
      src = os.path.splitext(name)
      os.rename(name,src[0]+'.png')

In [ ]:
images_directory = '/content/x_ray_dataset/images'

files = glob.glob(os.path.join(images_directory, '*.bmp'))

for name in files:
    if not os.path.isdir(name):
      src = os.path.splitext(name)
      os.rename(name,src[0]+'.png')

In [ ]:

root_dir = "/content/x_ray_dataset/annotation_file"
img_dir = os.path.join(root_dir, "images")

data = glob.glob(os.path.join(img_dir, "*.png"))

train = data[:200]
val = data[200:240]
test = data[240:]

# train.txt
with open(os.path.join(root_dir, "train.txt"), 'w') as f:
	f.write('\n'.join(train) + '\n')

# valid.txt
with open(os.path.join(root_dir, "valid.txt"), 'w') as f:
	f.write('\n'.join(val) + '\n')

# test.txt
with open(os.path.join(root_dir, "test.txt"), 'w') as f:
	f.write('\n'.join(test) + '\n')

In [ ]:
import yaml

yaml_data = {"names":['flaw'], # 클래스 이름
             "nc": 1, # 클래스 수
             "path":root_dir, # root 경로
             "train":os.path.join(root_dir, "train.txt"), # train.txt 경로
             "val":os.path.join(root_dir, "valid.txt"), # valid.txt 경로
             "test":os.path.join(root_dir,"test.txt") # test.txt 경로
             }

with open(os.path.join(root_dir, "custom.yaml"), "w") as f:
  yaml.dump(yaml_data, f)

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
%pip install -qr requirements.txt # install dependencies

Cloning into 'yolov5'...
remote: Enumerating objects: 16680, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 16680 (delta 102), reused 131 (delta 61), pack-reused 16459
Receiving objects: 100% (16680/16680), 15.37 MiB | 21.93 MiB/s, done.
Resolving deltas: 100% (11401/11401), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.6/793.6 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 65.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fo

In [ ]:
!python train.py --img 576 --batch 64 --epochs 100 --data /content/x_ray_dataset/annotation_file/custom.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name test --device 0

2024-06-27 07:08:10.309399: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 07:08:10.309465: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 07:08:10.411959: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/x_ray_dataset/annotation_file/custom.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=64, imgsz=576, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_wei

In [ ]:
!python val.py --weights /content/yolov5/runs/train/test/weights/best.pt --data /content/x_ray_dataset/annotation_file/custom.yaml --img 576 --task test

val: data=/content/x_ray_dataset/annotation_file/custom.yaml, weights=['/content/yolov5/runs/train/test/weights/best.pt'], batch_size=32, imgsz=576, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-331-gab364c98 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/x_ray_dataset/annotation_file/test... 60 images, 0 backgrounds, 0 corrupt: 100% 60/60 [00:00<00:00, 695.63it/s]
test: New cache created: /content/x_ray_dataset/annotation_file/test.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 2/2 [00:02<00:00,  1.37s/it]
                   all         60        122      0.984      0.983      0.